In [1]:
from tqdm import tqdm
import json
import pandas as pd
from pathlib import Path
import numpy as np

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

2025-06-16 17:50:35.715629: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750085435.727458   32654 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750085435.730931   32654 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750085435.741074   32654 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750085435.741091   32654 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750085435.741093   32654 computation_placer.cc:177] computation placer alr

In [2]:
split_dir = Path('dataset') / 'biobert_split'

In [3]:
X_train = np.load(split_dir / 'X_train.npy')
X_train_sent = np.load(split_dir / 'X_train_sent.npy')
X_test = np.load(split_dir / 'X_test.npy')
X_test_sent = np.load(split_dir / 'X_test_sent.npy')
y_train = np.load(split_dir / 'y_train.npy')
y_test = np.load(split_dir / 'y_test.npy')

len(X_train), len(X_test), len(y_train), len(y_test), len(X_train_sent), len(X_test_sent)

(338, 85, 338, 85, 338, 85)

In [4]:
X_train = [' '.join([w for w in row if w != '<PAD>']) for row in X_train]
X_test = [' '.join([w for w in row if w != '<PAD>']) for row in X_test]

X_train[0], X_test[0]

('em ##etophobia flying fly motion over the years husband 7 – 8 hour books 1 hour na husband regressed every day motion fly feel trapped',
 'intense emetophobia freaking out close boba tea stomach ache panic tired few hours later went to sleep woke up house ill fine eat stomach rumble almost 1 in the afternoon 30 minutes sick z ##ofran pep sick')

In [5]:
# sent_transformer = SentenceTransformer('allenai/scibert_scivocab_uncased')
sent_transformer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [6]:
X_train_sent_final = []
for row in X_train_sent:
    row = float(row)
    if row > 0.5:
        X_train_sent_final.append('positive')
    elif row < -0.5:
        X_train_sent_final.append('negative')
    else:
        X_train_sent_final.append('neutral')

X_test_sent_final = []
for row in X_test_sent:
    row = float(row)
    if row > 0.5:
        X_test_sent_final.append('positive')
    elif row < -0.5:
        X_test_sent_final.append('negative')
    else:
        X_test_sent_final.append('neutral')

X_train_sent_final[:5], X_test_sent_final[:5]

(['negative', 'negative', 'negative', 'negative', 'neutral'],
 ['negative', 'negative', 'neutral', 'negative', 'negative'])

In [7]:
X_train_final = []
for i in tqdm(range(len(X_train))):
    X_train_final.append(X_train[i] + ' ' + X_train_sent_final[i])

X_test_final = []
for i in tqdm(range(len(X_test))):
    X_test_final.append(X_test[i] + ' ' + X_test_sent_final[i])

X_train_final[:5], X_test_final[:5]

100%|██████████| 85/85 [00:00<00:00, 1310720.00it/s]


(['em ##etophobia flying fly motion over the years husband 7 – 8 hour books 1 hour na husband regressed every day motion fly feel trapped negative',
  'woke like 12am last night super throat body ache tu * scared d * stomach throat spray cough drop scared tu have an appetite everything hurts to eat negative',
  'anxiety time high anxiety physically upper chest fluttery feeling dia ##ragm panic attack heart attack chest left co ##rtisol adrenaline chest road trip 3 hours to breathe negative',
  'mom cart grandma healthy wednesday mind spiraling thoughts mom it kids nuts 2 yr old tu ##mmy hurt for a second anxiety negative',
  'it 3 bc brain painkillers left early weird fuzzy feeling cheeks bc scared ##ious spoiler 5 mins later gag it neutral'],
 ['intense emetophobia freaking out close boba tea stomach ache panic tired few hours later went to sleep woke up house ill fine eat stomach rumble almost 1 in the afternoon 30 minutes sick z ##ofran pep sick negative',
  'of stupidity ate 3 / 4 

In [8]:
n_clusters = len(['Question', 'Needing support - Panic attack', 'Rant', 'Potentially Triggering', 'Does Anyone Else...?', 'Needing support: Just not feeling good'])

In [9]:
topic_model = BERTopic(verbose=True, n_gram_range=(1, 2), min_topic_size=2, nr_topics=n_clusters, embedding_model=sent_transformer, calculate_probabilities=True)
topics, probs = topic_model.fit_transform(X_train_final)



2025-06-16 17:50:43,681 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

2025-06-16 17:50:44,008 - BERTopic - Embedding - Completed ✓
2025-06-16 17:50:44,008 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-16 17:50:51,223 - BERTopic - Dimensionality - Completed ✓
2025-06-16 17:50:51,224 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-16 17:50:51,255 - BERTopic - Cluster - Completed ✓
2025-06-16 17:50:51,256 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-06-16 17:50:51,295 - BERTopic - Representation - Completed ✓
2025-06-16 17:50:51,296 - BERTopic - Topic reduction - Reducing number of topics
2025-06-16 17:50:51,299 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-16 17:50:51,320 - BERTopic - Representation - Completed ✓
2025-06-16 17:50:51,321 - BERTopic - Topic reduction - Reduced number of topics from 45 to 6


In [10]:
np.unique(topics)

array([-1,  0,  1,  2,  3,  4])

In [11]:
from sklearn.metrics import silhouette_score

mask = np.array(topics) != -1 
topics_masked = np.array(topics)[mask]
X_train_final_masked = np.array(X_train_final)[mask]

embeddings = topic_model.embedding_model.embedding_model.encode(X_train_final_masked, show_progress_bar=True, convert_to_numpy=True)
silhouette = silhouette_score(embeddings, topics_masked)
print(f'Silhouette Score: {silhouette}')

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Silhouette Score: 0.038360416889190674
